In [1]:
import os
os.chdir('/scratch/sagarsj42/')

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
TRAIN_FILE = 'IRSE_FIRE_2022_Track_Training_Data_preprocessed.csv'
TEST_FILE = 'IRSE_FIRE_2022_Track_Test_Data_1000.csv'

In [4]:
irse_df = pd.read_csv(TRAIN_FILE)

print(irse_df.info())

irse_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8047 entries, 0 to 8046
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Comments                  8047 non-null   object
 1   Surrounding Code Context  8047 non-null   object
 2   Class                     8047 non-null   object
dtypes: object(3)
memory usage: 188.7+ KB
None


,Comments,Surrounding Code Context,Class
0,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
1,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful
2,/*done*/,"-10. multi_add_handle(m, curl);\n-9. for(;...",Not Useful
3,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
4,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful


In [5]:
test_df = pd.read_csv(TEST_FILE)

print(test_df.info())

test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Comments                  1001 non-null   object
 1   Surrounding Code Context  1001 non-null   object
 2   Class                     1001 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB
None


,Comments,Surrounding Code Context,Class
0,/*READ_INT_FUNCTIONS*/,-5. if (png_ptr != NULL)\n-4. png_war...,Not Useful
1,/*Put the chunk name into png_ptr->chunk_name.*/,"-2. png_read_data(png_ptr, buf, 8);\n-1. ...",Not Useful
2,/*critical*/,"-8. png_crc_read(png_structrp png_ptr, png_byt...",Not Useful
3,/*READ_iCCP|iTXt|pCAL|sCAL|sPLT|tEXt|zTXt|SEQU...,-9. {\n-8. if (warn != 0)\n-7. ...,Not Useful
4,/*ZLIB_VERNUM >= 0x1240*/,#if ZLIB_VERNUM >= 0x1240\n\n/*ZLIB_VERNUM >= ...,Not Useful


In [6]:
classes = sorted(irse_df['Class'].value_counts().index.tolist())

classes

['Not Useful', 'Useful']

In [7]:
irse_df['label'] = irse_df['Class'].apply(lambda l : classes.index(l))
test_df['label'] = test_df['Class'].apply(lambda l : classes.index(l))

irse_df.info(), test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8047 entries, 0 to 8046
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Comments                  8047 non-null   object
 1   Surrounding Code Context  8047 non-null   object
 2   Class                     8047 non-null   object
 3   label                     8047 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 251.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Comments                  1001 non-null   object
 1   Surrounding Code Context  1001 non-null   object
 2   Class                     1001 non-null   object
 3   label                     1001 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


(None, None)

In [8]:
irse_df = irse_df.drop_duplicates()

irse_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5949 entries, 0 to 8046
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Comments                  5949 non-null   object
 1   Surrounding Code Context  5949 non-null   object
 2   Class                     5949 non-null   object
 3   label                     5949 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 232.4+ KB


In [9]:
train_df, dev_df = train_test_split(irse_df, test_size=0.1, 
    stratify=irse_df['label'])

train_df.shape, dev_df.shape

((5354, 4), (595, 4))

In [10]:
train_df.to_csv('irse.train.csv', index=False)
dev_df.to_csv('irse.dev.csv', index=False)

In [11]:
pp_comments = irse_df['Comments']
pp_contexts = irse_df['Surrounding Code Context']
pp_labels = irse_df['label']
match_indexes = list()

for comment, context, label in zip(pp_comments, pp_contexts, pp_labels):
    match = test_df[(test_df['Comments'] == comment) & \
        (test_df['Surrounding Code Context'] == context)]
    match_indexes.extend(match.index.tolist())
    
    match_labels = match['label'].tolist()
    for ml in match_labels:
        assert ml == label
match_indexes = list(set(match_indexes))

len(match_indexes)

167

In [12]:
test_df_pure = test_df.drop_duplicates().drop(index=match_indexes, errors='ignore')

print(test_df_pure.info())

test_df_pure.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 678 entries, 0 to 1000
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Comments                  678 non-null    object
 1   Surrounding Code Context  678 non-null    object
 2   Class                     678 non-null    object
 3   label                     678 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 26.5+ KB
None


,Comments,Surrounding Code Context,Class,label
0,/*READ_INT_FUNCTIONS*/,-5. if (png_ptr != NULL)\n-4. png_war...,Not Useful,0
1,/*Put the chunk name into png_ptr->chunk_name.*/,"-2. png_read_data(png_ptr, buf, 8);\n-1. ...",Not Useful,0
2,/*critical*/,"-8. png_crc_read(png_structrp png_ptr, png_byt...",Not Useful,0
3,/*READ_iCCP|iTXt|pCAL|sCAL|sPLT|tEXt|zTXt|SEQU...,-9. {\n-8. if (warn != 0)\n-7. ...,Not Useful,0
4,/*ZLIB_VERNUM >= 0x1240*/,#if ZLIB_VERNUM >= 0x1240\n\n/*ZLIB_VERNUM >= ...,Not Useful,0


In [13]:
test_df_pure.to_csv('irse.test.csv', index=False)

In [ ]:
test_df.to_csv('irse.given-test.csv', index=False)